# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt

% matplotlib inline

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

## 1. What test is appropriate for this problem? Does CLT apply?

##### Note : The resume for both black and white sounding names are identical and ,as the data description suggests, the names were randomly assigned to the resume. This qualifies the " Randomization" requirement of the Central Limit Theorem.

In [3]:
# Display first 5 elements of the dataframe, data

data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [4]:
# Identifying column names of the dataframe, data

data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

In [5]:
# Separating the dataframe into black and white sounding names to explore dataset

data_b = data[data.race=='b']
data_w = data[data.race=='w']
num_b = len(data_b)
num_w = len(data_w)

print("Number of observations where name is black-sounding : {}  " .format(num_b))
print("Number of observations where name is white-sounding : {}  ".format(num_w))


Number of observations where name is black-sounding : 2435  
Number of observations where name is white-sounding : 2435  


##### Note : The number of observations ie. samples in the dataset are much smaller than the true black and white population, globally. This satisfies the "10% Rule" of CLT conditions 

In [6]:
# Number of call backs for both black and white sounding names

b_success = len(data_b[data_b.call == 1])
w_success = len(data_w[data_w.call == 1])

print("Number of black sounding names that got a callback :",b_success, " \nNumber of white sounding names that got a callback :",w_success)

Number of black sounding names that got a callback : 157  
Number of white sounding names that got a callback : 235


In [7]:
# Fraction of successful call backs for both black and white sounding names

p_b =  b_success/num_b
p_w =  w_success/num_w

print("Fraction of black sounding names getting a callback : ",p_b)
print("Fraction of white sounding names getting a callback : ",p_w)

Fraction of black sounding names getting a callback :  0.06447638603696099
Fraction of white sounding names getting a callback :  0.09650924024640657


In [8]:
# Evaluating applicability of Central Limit Theorem ie np>=10 and nq>=10

print(num_b * p_b)
print(num_b * (1-p_b))
print("---")
print(num_b * p_w)
print(num_b * (1-p_w))

157.0
2278.0
---
235.0
2200.0


##### Note : The sample size is large enough, as indicated by the results above. This test approved validity of Central Limit Theorem

 
### Central Limit Theorem is applicable because the following conditions are met :

1. Samples are random and independent
2. The sample size is less than 10% of the entire population
3. Sample size is large enough such that np >=10 and nq >= 10

### Appropriate Test Selection :

The population SD is not known. Two sample t test is appropriate to test the difference in response. 
  

## 2. What are the null and alternate hypotheses?

#### Ho : Average positive call back for the black sounding names = Average positive call back for the white sounding names

#### Ha : Average positive call back for the black sounding names is different from that for the white sounding names

In [9]:
# Performing two sample t test to evaluate p_value

from scipy.stats import ttest_ind

stat,p_value = ttest_ind(data[data.race=='b']['call'],data[data.race=='w']['call'],equal_var=False)

print(" p value :", p_value)
print(" Test statistic :", stat)

 p value : 3.94294151365e-05
 Test statistic : -4.11470529086


In [66]:
# Calls separated by name types - black and white sounding

data_b_call = data_b.call
data_w_call = data_w.call

# Defining a function to evaluate test statistic

def diff_of_means(data1,data2):
    
    return (np.mean(data1)-np.mean(data2))

# Test statistic for the actual data

obs_test_Stat = diff_of_means(data_w_call,data_b_call)

# Function to generate permutation replicates

def draw_perm_replicates(data1,data2,func,size=1):
    
    perm_replicates = np.empty(size)

    for i in range(size):
        perm_sample = np.random.permutation(np.concatenate((data1,data2)))
        perm_sample_data1 = perm_sample[:len(data1)]
        perm_sample_data2 = perm_sample[len(data1):]
        
        # Test statistic
        perm_replicates[i] = func(perm_sample_data1,perm_sample_data2)
        
    return perm_replicates

# Generating 10,000 permutation replicates

perm_replicates = draw_perm_replicates(data_w_call,data_b_call,diff_of_means,10000)

# p value
p_val = np.sum(perm_replicates >= obs_test_Stat) / len(perm_replicates)

print(" p value (bootstrap method) :", p_val)

print(" p value (t test) :", p_value)

 p value (bootstrap method) : 0.0
 p value (t test) : 3.94294151365e-05


##### Note :The p value is extremely small and sufficient to reject the null hypothesis

## 3. Margin of Error, Confidence Interval and p_value ?

In [10]:
# Reference : http://www.webassign.net/idcollabstat2/Chapter10.pdf

# Assume 95% confidence level

# Calculating Margin of error

SD_1 = np.std(data[data.race=='b']['call'])
SD_2 = np.std(data[data.race=='w']['call'])
n1 = len(data[data.race=='b'])
n2 = len(data[data.race=='w'])

std_error_of_difference = ((SD_1**2/n1) + (SD_2**2/n2))**0.5
degree_of_freedom = (((SD_1**2/n1) + (SD_2**2/n2))**2) /((1/(n1-1))*(((SD_1**2/n1))**2) + (1/(n2-1))*(((SD_2**2/n2))**2))

# critical t value selected from t-Tables (Ref : https://faculty.washington.edu/heagerty/Books/Biostatistics/TABLES/t-Tables/) 
t_c = 1.96

moe = t_c * std_error_of_difference

print(" Margin of Error :", moe)

 Margin of Error : 0.015255284385449893


In [11]:
# Confidence interval

mean_X1 = np.mean(data[data.race=='b']['call'])
mean_X2 = np.mean(data[data.race=='w']['call'])

lower = mean_X1 - mean_X2 - moe
upper = mean_X1 - mean_X2 + moe

print(" Confidence interval :", [lower,upper])


 Confidence interval : [-0.04728813924051047, -0.016777570469610682]


## 4. Write a story describe the statistical significance in the context of the original problem

The p-value is 3.9e-05, so we can reject the null hypothesis, which means we can say with confidence that people with black sounding names have the less chance of being called back as the people with white sounding names.

## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

No, the analysis does not suggest that race/name is the most important factor in callback success. It merely proves that race/name is one of many other factors that influence callback success. In order to determine the most important factor that affects callback success, all possible factors must be analyzed and ranked to test which feature is the most strongly related to callback.